# Análise exploratória

## Imports, definição de funções e carregamento dos dados

In [1]:
import manipular_banco
import preparar_texto

import preparar_texto
import matplotlib.pyplot as plt
import nltk
from nltk import ngrams

def mostrar_ngrams_frequentes(df, coluna, n):
    coluna_preparada = preparar_texto.prepar(df, coluna)
    todas_as_palavras = [palavra for palavras in df[coluna_preparada] for palavra in palavras]
    n_grams = ngrams(todas_as_palavras, n)
    nltk_freq_distr = nltk.FreqDist(n_grams)
    most_common = nltk_freq_distr.most_common(20)
    plt.barh(range(len(most_common)),[val[1] for val in most_common], align='center')
    plt.yticks(range(len(most_common)), [val[0] for val in most_common])
    plt.show()    
    
def mostrar_registros_contendo_str_apos_preparacao(df, coluna, ngram_as_str):
    coluna_preparada = preparar_texto.preparar(df, coluna)
    return df[df[coluna_preparada].str.contains(ngram_as_str)][coluna]
    
def calcular_tamanho(df, coluna, force = False):
    coluna_tamanho = 'tamanho_' + coluna
    if not coluna_tamanho in df or force:
        df['tamanho_' + coluna] = [len(c) for c in df[coluna]]
    return coluna_tamanho
    
def mostrar_histograma(df, coluna):
    coluna_tamanho = calcular_tamanho(df, coluna)
    plt.hist(df[coluna_tamanho])
    plt.show()
    
def mostrar_registros_com_tam_menor_q(df, coluna, tam):
    coluna_tamanho = calcular_tamanho(df, coluna)
    return df.loc[df[coluna_tamanho] < tam][coluna]
    
def mostrar_registros_com_tam_maior_q(df, coluna, tam):
    coluna_tamanho = calcular_tamanho(df, coluna)
    return df.loc[df[coluna_tamanho] > tam][coluna]

noticias_df = manipular_banco.carregar_noticias()

ModuleNotFoundError: No module named 'manipular_banco'

## Análise de palavras e ngrams

### Palavras mais frequentes do título

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'titulo', 1)

Das palavras muito frequentes acima, "veja" parece indicar matérias desprovidas de utilidade. Necessário analisar os bigrams.

### Bigrams mais frequentes do título

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'titulo', 2)

Dos bigrams acima, parece estranho "vagas emprego", "homem preso", "morre apos", "neste sabado", "veja valores", "segundafeira maio" (ou "sextafeira maio") constarem entre os mais frequentes. Verificando cada expressão dessas...

#### "vagas emprego"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'vagas emprego')

**Decisão:** Algumas das matérias contendo o bigram "vagas emprego" parecem ser publicadas diariamente, e não tratar de assuntos específicos, cuja cobertura possa variar com o tempo. É necessário analisar os trigrams para decidir manter ou eliminar.

#### "homem preso"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'homem preso')

**Decisão:** ignorar "homem preso" pois parece apenas uma expressão frequente, em notícias legítimas

#### "morre apos"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'morre apos')

**Decisão:** ignorar "morre apos" pois parece apenas uma expressão frequente, em notícias legítimas

#### "neste sabado"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'neste sabado')

**Decisão:** ignorar "neste sabado" pois parece apenas uma expressão frequente, em notícias legítimas

#### "veja valores"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'veja valores')

**Decisão:** remover as matérias que contêm o bigram "veja valores" no título pois parecem ser publicadas diariamente, e não tratar de assuntos específicos, cuja cobertura possa variar com o tempo.

In [ ]:
remover_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'veja valores')

#### "segundafeira maio"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'segundafeira maio')

**Decisão:** remover as matérias cujo título começa com "VÍDEOS:" pois parecem ser publicadas diariamente, e não tratar de assuntos específicos, cuja cobertura possa variar com o tempo.

In [ ]:
remover_registros_contendo_regex(noticias_df, 'titulo', '^VÍDEOS:')

### Trigrams mais frequentes do título

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'titulo', 3)

Dos trigrams acima, considerando também o resultado da análise dos bigrams, parece estranho "apos dois anos", "veja boletim prefeitura", "covid veja boletim" constarem entre os mais frequentes.

Também causam estranheza as expressões "araripina salgueiro nesta", "petrolina araripina salgueiro", "disponiveis petrolina araripina", "emprego disponiveis petrolina", "ofertas vagas emprego", "vagas emprego nesta", "veja vagas emprego". 

Verificando cada expressão dessas...

#### "apos dois anos"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'apos dois anos')

**Decisão:** ignorar "apos dois anos" pois parece apenas uma expressão frequente, em notícias legítimas

#### "veja boletim prefeitura" e "covid veja boletim"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'veja boletim prefeitura')

**Decisão:** remover as matérias cujo título começa com "Covid-19: veja boletim" pois, embora tratem de um acontecimento específico, parecem ser publicadas diariamente no mesmo formato.

In [ ]:
remover_registros_contendo_regex(noticias_df, 'titulo', '^Covid-19: veja boletim')

#### "veja vagas emprego", "vagas emprego nesta", "emprego disponiveis petrolina", "disponiveis petrolina araripina", "petrolina araripina salgueiro" e "araripina salgueiro nesta"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'veja vagas emprego')

**Decisão:** remover as matérias cujo título contém a expressão regular "veja .*vagas de emprego", pois parecem ser publicadas diariamente, e não tratar de assuntos específicos, cuja cobertura possa variar com o tempo.

In [ ]:
remover_registros_contendo_regex(noticias_df, 'titulo', 'eja .*vagas de emprego')

#### "oferta vagas emprego"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'oferta vagas emprego')

**Decisão:** remover as matérias que contêm o trigram "oferta vagas emprego" no título pois parecem ser publicadas diariamente, e não tratar de assuntos específicos, cuja cobertura possa variar com o tempo.

In [ ]:
remover_registros_contendo_str_apos_preparacao(noticias_df, 'titulo', 'oferta vagas emprego')

### Palavras mais frequentes do resumo

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'resumo', 1)

Das palavras muito frequentes acima, "veja" e "videos" parecem indicar resumos desprovidos de utilidade. Necessário analisar os bigrams.

### Bigrams mais frequentes do resumo

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'resumo', 2)

Dos bigrams acima, e considerando a análise feita anteriormente sobre o resumo, parece estranho "noticias regiao", "tudo sobre", "veja noticias", "mix vale", "primeiro mix" e "apareceu primeiro" constarem entre os mais frequentes. Verificando cada expressão dessas...

#### "noticias regiao"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'noticias regiao')

**Decisão:** remover do fim do resumo as frases que iniciam em "Confira mais notícias", "Veja mais notícias", "VÍDEOS:" ou "Assista aos vídeos" pois podem contribuir para indicar similaridade entre matérias totalmente diferentes

In [ ]:
remover_trechos_contendo_regex(noticias_df, 'resumo', '(?:\.|\\n).*?(?:Veja|Confira|Assista|Saiba|Leia|Ouça|Compartilhe).*?(?:\.|\\n|$)')
remover_trechos_contendo_regex(noticias_df, 'resumo', '(?:\.|\\n).*?vídeos.*?(?:\.|\\n|$)')

#### "tudo sobre"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'tudo sobre')

**Decisão:** remover do resumo frases que contenham "Tudo sobre"

In [ ]:
remover_trechos_contendo_regex(noticias_df, 'resumo', '(?:\.|\\n).*?udo sobre.*?(?:\.|\\n|$)')

#### "veja noticias"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'veja noticias')

Sem ocorrências, depois de executados os procedimentos referentes aos bigrams acima

#### "apareceu primeiro", "primeiro mix" e "mix vale"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'mix vale')

**Decisão:** eliminar a frase "apareceu primeiro no Mix Vale."

In [ ]:
remover_trechos_contendo_regex(noticias_df, 'resumo', '(?:\.|\\n).*?apareceu primeiro.*?(?:\.|\\n|$)')

### Trigrams mais frequentes do resumo

In [ ]:
mostrar_ngrams_frequentes(noticias_df, 'resumo', 3)

Dos trigrams acima, parece estranho "getty images via", "reproducao tv globo", "medio completo experiencia" (e "ensino médio completo"), "primeiro revista oeste" (e "apareceu primeiro jornal") constarem entre os mais frequentes.

Verificando cada expressão dessas...

#### getty images via

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'getty images via')

**Decisão:** remover a frase "Getty Images via BBC"

In [ ]:
remover_trechos_contendo_regex(noticias_df, 'resumo', 'Getty Images.*?(\\n|\.)')

#### "reproducao tv globo"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'reproducao tv globo')

**Decisão:** remover a frase "Reprodução/TV Globo"

In [ ]:
remover_trechos_contendo_regex(noticias_df, 'resumo', 'Reprodução\s*/\s*TV Globo')

#### "medio completo experiencia"

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'medio completo experiencia')

**Decisão:** remover matérias contendo o bigram "completo experiencia" pois...

In [ ]:
remover_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'completo experiencia')

#### "primeiro revista oeste" (e "apareceu primeiro jornal")

In [ ]:
mostrar_registros_contendo_str_apos_preparacao(noticias_df, 'resumo', 'compartilhe')

## Análise dos textos quanto ao tamanho

### Título

#### Histograma do título

In [ ]:
mostrar_histograma(noticias_df, 'titulo')

Não parece haver uma variação de tamanho muito discrepante entre os títulos. Para confirmar, analisar os títulos cujo tamanho está nas faixas extremas do histograma:

#### Títulos muito curtos

In [ ]:
mostrar_registros_com_tam_menor_q(noticias_df, 'titulo', 21)

Seria proveitoso apenas verificar do que se tratam as matérias cujo título tem apenas uma palavra:

In [ ]:
noticias_df[noticias_df['titulo'].isin(['Posse', 'Resgate', 'Aurora'])]

**Decisão:** manter todas essas matérias, pois parecem legítimas

#### Títulos muito longos

In [ ]:
mostrar_registros_com_tam_maior_q(noticias_df, 'titulo', 140)

**Decisão:** Todas essas parecem matérias legítimas, sem necessidade de intervenção

### Resumo

#### Matérias sem resumo

In [ ]:
len(noticias_df[noticias_df['resumo'] == ''])

In [ ]:
len(noticias_df[noticias_df['resumo'] == None])

In [ ]:
len(noticias_df[noticias_df['resumo'].isnull()])

Pelos comandos acima, verifica-se que há um número considerável matérias com resumo vazio. O impacto disso terá de ser obvervado ao se avaliar a execução dos algoritmos de machine learning.

#### Histograma do resumo

In [ ]:
mostrar_histograma(noticias_df, 'resumo')

A grande maioria dos resumos tem tamanho menor que 3500 caracteres. Percebe-se uma amplitude considerável nos tamanhos dos resumos. Ou seja, há uma variação muito grande na quantidade de informação contida nos resumos. Assim como mencionado com relação à ausência do resumo, o impacto dessa variação terá de ser obvervado ao se avaliar a execução dos algoritmos de machine learning.